<a href="https://colab.research.google.com/github/moonstar0301/Corn_Future_Prediction/blob/main/VECM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/CORNdataset.xlsx', index_col=0)
data = df

In [3]:
train_data = data.loc['2002-01':'2017-12']
test_data = data.loc['2018-01':'2021-12']

In [4]:
train_data

,NonComm_Positions_Long_All,SOI,WTI종가,금리,달러화가치,옥수수선물
Month,,,,,,
2002-01,100274.75,1.5,19.48,0.70,120.590,213.00
2002-02,91036.75,2.8,21.74,1.29,119.280,207.50
2002-03,72378.00,-2.5,26.31,-4.48,119.060,209.00
2002-04,66511.20,-1.0,27.29,3.04,115.400,200.50
2002-05,75927.50,-3.7,25.31,1.11,111.910,220.50
...,...,...,...,...,...,...
2017-08,392398.00,0.7,47.23,1.08,92.628,342.25
2017-09,375778.75,2.3,51.67,-1.52,92.883,355.25
2017-10,377928.00,2.3,54.38,-0.30,94.429,345.75


In [5]:
import pandas as pd
from statsmodels.tsa.api import VAR

results = []
for i in range(1, 13):
    model = VAR(train_data)
    results.append(model.fit(i).aic)
    results.append(model.fit(i).fpe)
    results.append(model.fit(i).hqic)
    results.append(model.fit(i).bic)

results = np.array(results).reshape(12, 4)
results_df = pd.DataFrame(results, index=range(1, 13), columns=['AIC', 'FPE', 'HQ', 'SC'])

print(results_df)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_date

          AIC           FPE         HQ         SC
1   36.165915  5.090294e+15  36.455588  36.881074
2   35.763476  3.407519e+15  36.303450  37.096465
3   35.851964  3.733240e+15  36.644119  37.807304
4   36.110714  4.861977e+15  37.156952  38.692982
5   36.217362  5.458178e+15  37.519607  39.431192
6   36.343828  6.279608e+15  37.904029  40.193913
7   36.353028  6.463517e+15  38.173157  40.844119
8   36.493152  7.639297e+15  38.575205  41.630060
9   36.555255  8.426346e+15  38.901252  42.342852
10  36.686917  1.007229e+16  39.298906  43.130140
11  36.724558  1.110255e+16  39.604609  43.828402
12  36.801531  1.292872e+16  39.951740  44.571059


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [6]:
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.vecm import VECM
from sklearn.preprocessing import MinMaxScaler

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 정규화
scaler = MinMaxScaler()
train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data), 
                                 columns=train_data.columns, index=train_data.index)
test_data_scaled = pd.DataFrame(scaler.transform(test_data), 
                                columns=test_data.columns, index=test_data.index)

# cointegration test
johan_test_result = coint_johansen(train_data_scaled,-1,1)
traces = johan_test_result.lr1
crit_values = johan_test_result.cvt[:,1]

# 고유 벡터 계산
eigenvectors = johan_test_result.evec

# cointegration rank 결정
cointegration_rank = 0
for i in range(len(traces)):
    if traces[i] > crit_values[i]:
        cointegration_rank += 1
        
# VECM 모델 학습
model = VECM(train_data_scaled, k_ar_diff=3, coint_rank=cointegration_rank, deterministic="co")
vecm_fit = model.fit()

# test_data 예측
test_data_pred = vecm_fit.predict(steps=len(test_data_scaled))

# 정규화 된 예측값을 다시 원래 스케일로 되돌리기
test_data_pred = pd.DataFrame(scaler.inverse_transform(test_data_pred), 
                              columns=test_data.columns, index=test_data.index)

# MAPE 계산
mape_result = mape(test_data.values, test_data_pred.values)

mape_result

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


49.44725639811715

In [8]:
train_data = train_data.drop("NonComm_Positions_Long_All", axis=1)
test_data = test_data.drop("NonComm_Positions_Long_All", axis=1)

In [9]:
train_data

,SOI,WTI종가,금리,달러화가치,옥수수선물
Month,,,,,
2002-01,1.5,19.48,0.70,120.590,213.00
2002-02,2.8,21.74,1.29,119.280,207.50
2002-03,-2.5,26.31,-4.48,119.060,209.00
2002-04,-1.0,27.29,3.04,115.400,200.50
2002-05,-3.7,25.31,1.11,111.910,220.50
...,...,...,...,...,...
2017-08,0.7,47.23,1.08,92.628,342.25
2017-09,2.3,51.67,-1.52,92.883,355.25
2017-10,2.3,54.38,-0.30,94.429,345.75


In [10]:
import pandas as pd
from statsmodels.tsa.api import VAR

results = []
for i in range(1, 13):
    model = VAR(train_data)
    results.append(model.fit(i).aic)
    results.append(model.fit(i).fpe)
    results.append(model.fit(i).hqic)
    results.append(model.fit(i).bic)

results = np.array(results).reshape(12, 4)
results_df = pd.DataFrame(results, index=range(1, 13), columns=['AIC', 'FPE', 'HQ', 'SC'])

print(results_df)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_date

          AIC           FPE         HQ         SC
1   15.316386  4.486076e+06  15.523295  15.827214
2   15.169184  3.874131e+06  15.549935  16.109112
3   15.177667  3.912543e+06  15.733565  16.549835
4   15.360150  4.708271e+06  16.092517  17.167738
5   15.487975  5.373767e+06  16.398146  17.734200
6   15.611638  6.121589e+06  16.700967  18.299760
7   15.703628  6.775095e+06  16.973485  18.836947
8   15.800255  7.559180e+06  17.252027  19.382113
9   15.808782  7.754826e+06  17.443871  19.842562
10  15.923221  8.888143e+06  17.743049  20.412351
11  15.845935  8.459025e+06  17.851940  20.793886
12  15.895200  9.199672e+06  18.088839  21.305488


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
